In [19]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM, GlobalMaxPooling1D, Dropout
from tensorflow.keras import regularizers
from sklearn.metrics import classification_report
import pickle

def read_tbl():
    df = pd.read_csv("../../label/data/output/output.csv")
    return df

df = read_tbl()

# Preprocess the data
texts = df['gist'].astype(str).tolist()
labels = df['label'].astype(int).tolist()

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index

maxlen = 100
data = pad_sequences(sequences, maxlen=maxlen)
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Define the neural network
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1, output_dim=100, input_length=maxlen))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01), bias_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=16, validation_split=0.2, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f'Test Accuracy: {accuracy:.4f}')

y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype("int32")
print(classification_report(y_test, y_pred_classes))

model.save('../data/output/gist_classification_model.h5')

with open('../data/output/tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

Epoch 1/30
19/19 [==============================] - 0s 6ms/step - loss: 1.7409 - accuracy: 0.5400 - val_loss: 1.6030 - val_accuracy: 0.5200
Epoch 2/30
19/19 [==============================] - 0s 2ms/step - loss: 1.4984 - accuracy: 0.5500 - val_loss: 1.3815 - val_accuracy: 0.5200
Epoch 3/30
19/19 [==============================] - 0s 2ms/step - loss: 1.2956 - accuracy: 0.6133 - val_loss: 1.2064 - val_accuracy: 0.5200
Epoch 4/30
19/19 [==============================] - 0s 2ms/step - loss: 1.1351 - accuracy: 0.6667 - val_loss: 1.0661 - val_accuracy: 0.6000
Epoch 5/30
19/19 [==============================] - 0s 3ms/step - loss: 1.0026 - accuracy: 0.7133 - val_loss: 0.9416 - val_accuracy: 0.9200
Epoch 6/30
19/19 [==============================] - 0s 2ms/step - loss: 0.8800 - accuracy: 0.8933 - val_loss: 0.8139 - val_accuracy: 0.9733
Epoch 7/30
19/19 [==============================] - 0s 2ms/step - loss: 0.7384 - accuracy: 0.9233 - val_loss: 0.6502 - val_accuracy: 0.9867
Epoch 8/30
19/19 [==

/opt/homebrew/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import pickle
from transformers import BertTokenizer, TFBertModel

def read_tbl():
    df = pd.read_csv("../../label/data/output/output.csv")
    return df

df = read_tbl()

# Preprocess the data
texts = df['gist'].astype(str).tolist()
labels = df['label'].astype(int).tolist()

# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Tokenize the texts and create input sequences
input_ids = []
attention_masks = []

for text in texts:
    encoded_dict = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_tensors='tf'
    )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = np.concatenate(input_ids, axis=0)
attention_masks = np.concatenate(attention_masks, axis=0)

# Generate BERT embeddings
bert_outputs = bert_model(input_ids, attention_mask=attention_masks)
X = bert_outputs.last_hidden_state[:, 0, :].numpy()

X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

# Define and train the logistic regression model
lr_model = LogisticRegression(C=10, solver='liblinear', random_state=42)
lr_model.fit(X_train, y_train)

# Evaluate the logistic regression model
lr_y_pred = lr_model.predict(X_test)
print("Logistic Regression Classification Report:")
print(classification_report(y_test, lr_y_pred))

# Define and train the random forest model
rf_model = RandomForestClassifier(n_estimators=500, max_depth=10, min_samples_split=5, min_samples_leaf=2, random_state=42)
rf_model.fit(X_train, y_train)

# Evaluate the random forest model
rf_y_pred = rf_model.predict(X_test)
print("Random Forest Classification Report:")
print(classification_report(y_test, rf_y_pred))

# Save the models and tokenizer
with open('../data/output/gist_classification_lr_model.pkl', 'wb') as handle:
    pickle.dump(lr_model, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../data/output/gist_classification_rf_model.pkl', 'wb') as handle:
    pickle.dump(rf_model, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../data/output/bert_tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.90      0.87        48
           1       0.88      0.83      0.85        46

    accuracy                           0.86        94
   macro avg       0.86      0.86      0.86        94
weighted avg       0.86      0.86      0.86        94

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.72      0.85      0.78        48
           1       0.81      0.65      0.72        46

    accuracy                           0.76        94
   macro avg       0.77      0.75      0.75        94
weighted avg       0.76      0.76      0.75        94

